In [ ]:
%cd C:/Users/MELODY/Documents/Forex/ICT/ICT_ml_trading


In [ ]:
%pip install -e .


In [ ]:
%pip install -e .

from src.utils.visualization import plot_equity_curve, plot_drawdown, plot_metric_bar
import pandas as pd, numpy as np
import os, sys
print("cwd:", os.getcwd())
print("python:", sys.executable)
dates = pd.date_range('2025-01-01', periods=10, freq='D')
eq = pd.Series(1000 * (1 + np.linspace(0, 0.1, 10)), index=dates)
plot_equity_curve(eq, show=True)
plot_drawdown(eq, show=True)
plot_metric_bar({'sharpe_ratio': 1.23, 'max_drawdown': -0.05}, show=True)


In [10]:
# run_pipeline.py

import os
import sys
from pathlib import Path

# ────────────────────────────────────────────────────────
# 1) Make your `src/` folder importable without packaging
PROJECT_ROOT = Path(os.getcwd()).resolve()
SRC_DIR      = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))
# ────────────────────────────────────────────────────────

# 2) Imports (exactly as your tests use them)
from data_processing.data_loader    import DataLoader
from features.feature_engineering   import ICTFeatureEngineer
from ml_models.model_builder        import build_random_forest
from ml_models.trainer              import grid_search_with_checkpoint
from trading.strategy               import TradingStrategy
from trading.backtester             import backtest_signals

# 3) Load your **local** CSVs from `data/` (not `data/raw/`)
loader = DataLoader(data_path=PROJECT_ROOT / "data")

# Ensure the local file exists before loading
local_file = PROJECT_ROOT / "data" / "GBPUSD=X_1hr.csv"
if not local_file.exists():
    raise FileNotFoundError(f"Local data file not found: {local_file}")

df = loader.load_data(
    symbol="GBPUSD",
    start_date="2023-01-01",
    end_date="2023-01-10",
    interval="1h",
    data_source="local"   # force local-only
)

# 4) Feature engineering
fe = ICTFeatureEngineer(lookback_periods=[5,10,20], feature_selection_threshold=0.01)
fs = fe.engineer_features(df, symbol="EURUSD", additional_data={})
X = fs.features
y = X.pop("future_direction_5")  # your target

# 5) Train a tiny grid (uses your existing checkpoint logic)
model, grid_results = grid_search_with_checkpoint(
    build_random_forest(n_estimators=50, pca_components=None),
    {"clf__n_estimators": [50, 100]},
    X, y,
    cv=3,
    scoring="accuracy",
    checkpoint_path=PROJECT_ROOT / "checkpoints" / "rf_checkpoint.pkl"
)
print("Best model params:", model.get_params())

# 6) Generate signals & backtest
signals = TradingStrategy(model).generate_signals(X)
equity  = backtest_signals(signals, df[["close"]], initial_equity=10_000)
print("Final equity:", equity.iloc[-1])

# 7) Save equity curve for inspection
(out := PROJECT_ROOT / "reports").mkdir(exist_ok=True)
equity.to_csv(out / "equity_curve.csv")
print("Equity curve →", out / "equity_curve.csv")


FileNotFoundError: Local data file not found: C:\Users\MELODY\AppData\Local\Programs\Microsoft VS Code\data\GBPUSD=X_1hr.csv